## Image classification

In [12]:
import os
PATH = os.getcwd()
PATH

'D:\\'

## List of folders

In [13]:
DATA_PATH = os.path.join(PATH, 'Yoga')
data_dir_list = os.listdir(DATA_PATH)

In [14]:
data_dir_list

['Adho Mukha Svanasana',
 'Ashtanga Namaskara',
 'Ashwa Sanchalasana',
 'Bhujangasana',
 'Chaturanga Dandasana',
 'Hasta Uttanasana',
 'Pada Hastasana',
 'Tadasana',
 'Trikonasna',
 'Vrikshasana']

In [15]:
img_rows=224
img_cols=224
num_channel=3


img_data_list=[]
classes_names_list=[]

## Reading the data

In [16]:
import cv2

for dataset in data_dir_list:
    classes_names_list.append(dataset) 
    print ('Loading images from {} folder\n'.format(dataset)) 
    img_list=os.listdir(DATA_PATH+'/'+ dataset)
    for img in img_list:
        input_img=cv2.imread(DATA_PATH + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(img_rows, img_cols))
        img_data_list.append(input_img_resize)

Loading images from Adho Mukha Svanasana folder

Loading images from Ashtanga Namaskara folder

Loading images from Ashwa Sanchalasana folder

Loading images from Bhujangasana folder

Loading images from Chaturanga Dandasana folder

Loading images from Hasta Uttanasana folder

Loading images from Pada Hastasana folder

Loading images from Tadasana folder

Loading images from Trikonasna folder

Loading images from Vrikshasana folder



In [18]:
num_classes = len(classes_names_list)
print(num_classes)

10


In [19]:
import numpy as np

img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255

In [20]:
print (img_data.shape)

(640, 224, 224, 3)


In [22]:
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape

In [23]:
classes = np.ones((num_of_samples,), dtype='int64')

classes[0:56]=0
classes[56:115]=1
classes[115:175]=2
classes[175:235]=3
classes[235:295]=4
classes[295:355]=5
classes[355:455]=6
classes[455:485]=7
classes[485:544]=8
classes[544:]=9


## Converting into one-hot- encoding

In [24]:
from keras.utils import to_categorical

classes = to_categorical(classes, num_classes)

## Shuffling the data

In [25]:
from sklearn.utils import shuffle

X, Y = shuffle(img_data, classes, random_state=2)

## train-Test-Split

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [27]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

## Modeling

In [37]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [40]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])

In [41]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 220, 220, 32)      9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 110, 110, 32)      0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 108, 108, 64)      18496     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 106, 106, 64)      36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 53, 53, 64)       

In [44]:
hist = model.fit(X_train, y_train, batch_size=64, epochs=100, verbose=1, validation_data=(X_test, y_test))

Train on 512 samples, validate on 128 samples
Epoch 1/100
512/512 [==============================] - 77s 150ms/step - loss: 2.2840 - accuracy: 0.1426 - val_loss: 2.2962 - val_accuracy: 0.1094
Epoch 2/100
128/512 [======>.......................] - ETA: 49s - loss: 2.3455 - accuracy: 0.0938

KeyboardInterrupt: 

In [ ]:
print(model.predict(test_image))
print(model.predict_classes(test_image))
print(y_test[0:1])

## Predicting on test data

In [ ]:
from sklearn.metrics import confusion_matrix

Y_pred = model.predict(X_test)
print(Y_pred)

In [ ]:
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)

In [ ]:
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size)

print('Test Loss:', score[0])
print('Test Accuracy:', score[1])

## Custom training using VGGnet

In [45]:
from keras.layers import Input, Dense

In [46]:
image_input = Input(shape=(img_rows, img_cols, num_channel))

In [47]:
from keras.applications.vgg16 import VGG16

model = VGG16(input_tensor=image_input, include_top=True, weights='imagenet')

553467904/553467096 [==============================] - 435s 1us/step


In [48]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [49]:
last_layer = model.get_layer('fc2').output
out = Dense(num_classes, activation='softmax', name='output')(last_layer)

In [50]:
from keras.models import Model

custom_vgg_model = Model(image_input, out)
custom_vgg_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [52]:
custom_vgg_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


In [53]:
custom_vgg_model.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1, validation_data=(X_test, y_test))

Train on 512 samples, validate on 128 samples
Epoch 1/100
256/512 [==============>...............] - ETA: 8:02 - loss: 60595057.5770 - accuracy: 0.0703

KeyboardInterrupt: 

In [ ]:
Y_train_pred = custom_vgg_model.predict(X_test)

In [ ]:
y_train_pred = np.argmax(Y_train_pred, axis=1)
print(y_train_pred)

In [ ]:
print(confusion_matrix(np.argmax(y_test, axis=1), y_train_pred))